In [1]:
from pyspark.sql import SparkSession, types, functions as F

In [2]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('events_etl')\
  .getOrCreate()

25/04/16 10:25:07 WARN Utils: Your hostname, Bastiens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.243.218.34 instead (on interface en0)
25/04/16 10:25:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 10:25:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_nested = spark.read\
  .option('multiline', 'true')\
  .json('./data/events')

In [11]:
df_nested.show(5)

+-----+-------------+------------+-------------+-----+-----+---------+------------+-------+----+--------+--------------+--------+----------+--------+----------+--------------------+-----+---------------+------------+------------+------+----------+----------+----+--------------------+------+------------------+--------------------+----------+--------------------+----------+---------------+--------------------+------+----+------------+--------------------+-------------+------------+-----------------+--------------+
|50_50|bad_behaviour|ball_receipt|ball_recovery|block|carry|clearance|counterpress|dribble|duel|duration|foul_committed|foul_won|goalkeeper|half_end|half_start|                  id|index|injury_stoppage|interception|    location|minute|miscontrol|off_camera| out|                pass|period|      play_pattern|              player|player_off|            position|possession|possession_team|      related_events|second|shot|substitution|             tactics|         team|   timestam

In [ ]:
df_flat = df_nested.select(
  F.col('id').cast(types.StringType()),
  F.col('index').cast(types.IntegerType()),
  F.col('period').cast(types.IntegerType()),
  F.col('timestamp').cast(types.TimestampType()),
  F.col('second').cast(types.IntegerType()),
  F.col('duration').cast(types.FloatType()),
  # PLAY PATTERN
  F.col('play_pattern.id').cast(types.IntegerType()).alias('play_pattern_id'),
  F.col('play_pattern.name').cast(types.StringType()).alias('play_pattern'),
  F.col('type.id').cast(types.IntegerType()).alias('event_type_id'),
  F.col('type.name').cast(types.StringType()).alias('event_type_name'),
  # PLAYER
  F.col('player.id').cast(types.IntegerType()).alias('player_id'),
  F.col('player.name').cast(types.StringType()).alias('player'),
  F.col('position.id').cast(types.IntegerType()).alias('player_position_id'),
  F.col('position.name').cast(types.StringType()).alias('player_position'),
  # PLAY TYPES
  F.col('50_50.outcome.id').cast(types.IntegerType()).alias('50_50_id'),
  F.col('50_50.outcome.name').cast(types.StringType()).alias('50_50_outcome'),
  F.col('bad_behaviour.card.id').cast(types.IntegerType()).alias('card_id'),
  F.col('bad_behaviour.card.name').cast(types.StringType()).alias('card'),
  F.col('ball_receipt.outcome.id').cast(types.IntegerType()).alias('ball_receipt_outcome_id'),
  F.col('ball_receipt.outcome.name').cast(types.StringType()).alias('ball_receipt_outcome'),
  F.col('ball_recovery.offensive').cast(types.BooleanType()).alias('ball_recovery_offensive'),
  F.col('ball_recovery.recovery_failure').cast(types.BooleanType()).alias('ball_recovery_failure'),
  F.col('block.deflection').cast(types.BooleanType()).alias('block_deflection'),
  F.col('block.offensive').cast(types.BooleanType()).alias('block_offensive'),
  F.col('block.save_block').cast(types.BooleanType()).alias('block_save'),
  F.col('clearance.aerial_won').cast(types.BooleanType()).alias('clearance_aerial_won'),
  F.col('clearance.body_part.id').cast(types.IntegerType()).alias('clearance_body_part_id'),
  F.col('clearance.body_part.name').cast(types.StringType()).alias('clearance_body_part'),
  F.col('clearance.head').cast(types.BooleanType()).alias('clearance_head'),
  F.col('clearance.left_foot').cast(types.BooleanType()).alias('clearance_left_foot'),
  F.col('clearance.right_foot').cast(types.BooleanType()).alias('clearance_right_foot'),
  F.col('clearance.other').cast(types.BooleanType()).alias('clearance_other'),
  F.col('counterpress').cast(types.BooleanType()),
  F.col('dribble.no_touch').cast(types.BooleanType()).alias('dribble_no_touch'),
  F.col('dribble.nutmeg').cast(types.BooleanType()).alias('dribble_nutmeg'),
  F.col('dribble.overrun').cast(types.BooleanType()).alias('dribble_overrun'),
  F.col('dribble.outcome.id').astype(types.IntegerType()).alias('dribble_id'),
  F.col('dribble.outcome.name').astype(types.StringType()).alias('dribble_name'),
  F.col('duel.outcome.id').astype(types.IntegerType()).alias('duel_outcome_id'),
  F.col('duel.outcome.name').astype(types.StringType()).alias('duel_outcome'),
  F.col('duel.type.id').astype(types.IntegerType()).alias('duel_type_id'),
  F.col('duel.type.name').astype(types.StringType()).alias('duel_type_name'),
  # F.col('player_off').cast(types.BooleanType()),
  F.col('possession').cast(types.IntegerType()),
  # TEAM
  F.col('possession_team.id').cast(types.IntegerType()).alias('possession_team_id'),
  F.col('possession_team.name').cast(types.StringType()).alias('possession_team_name'),
  F.col('team.id').cast(types.IntegerType()).alias('event_team_id'),
  F.col('team.name').cast(types.StringType()).alias('event_team_name'),
  F.col('under_pressure').cast(types.BooleanType())
)

In [30]:
df_flat.show()

+--------------------+-----+------+--------------------+------+--------+-------------+---------------+---------------+-------------+---------+--------------------+------------------+--------------------+--------+-------------+-------+----+-----------------------+--------------------+-----------------------+---------------------+----------------+---------------+----------+--------------------+----------------------+-------------------+--------------+-------------------+--------------------+---------------+------------+----------------+--------------+---------------+----------+------------+---------------+------------+------------+--------------+----------+------------------+--------------------+-------------+---------------+--------------+
|                  id|index|period|           timestamp|second|duration|event_type_id|event_type_name|play_pattern_id| play_pattern|player_id|              player|player_position_id|     player_position|50_50_id|50_50_outcome|card_id|card|ball_receipt

In [4]:
df_nested.printSchema()

root
 |-- 50_50: struct (nullable = true)
 |    |-- outcome: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- bad_behaviour: struct (nullable = true)
 |    |-- card: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- ball_receipt: struct (nullable = true)
 |    |-- outcome: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- ball_recovery: struct (nullable = true)
 |    |-- offensive: boolean (nullable = true)
 |    |-- recovery_failure: boolean (nullable = true)
 |-- block: struct (nullable = true)
 |    |-- deflection: boolean (nullable = true)
 |    |-- offensive: boolean (nullable = true)
 |    |-- save_block: boolean (nullable = true)
 |-- carry: struct (nullable = true)
 |    |-- end_location: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- clearan